In [ ]:
# ────────────────────────────────────────────────
# ❶ 依存ライブラリのインストール（最初だけ）
# ────────────────────────────────────────────────
!pip -q install --upgrade faiss-cpu sentence_transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.0 MB/s eta 0:00:00


In [ ]:
# ────────────────────────────────────────────────
# ❷ Google Drive をマウント
# ────────────────────────────────────────────────
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# ────────────────────────────────────────────────
# ❸ インポート
# ────────────────────────────────────────────────
import os, numpy as np, matplotlib.pyplot as plt
from glob import glob
from pathlib import Path
from PIL import Image
import faiss, torch
from sentence_transformers import SentenceTransformer

# ★★ 自分の環境に合わせてここだけ書き換えてください ★★
IMAGES_PATH = '/content/drive/MyDrive/images'        # 画像が入った最上位フォルダ
INDEX_PATH  = '/content/drive/MyDrive/faiss_indexes/vector.index'
TOP_K       = 5   #似ている画像を上位何件検索するかの設定です。デフォルトは5にしていますが変えても大丈夫です


In [ ]:
# ────────────────────────────────────────────────
# ❹ CLIP モデルをロード
# ────────────────────────────────────────────────
device = 'cpu'
model  = SentenceTransformer('clip-ViT-B-32', device=device)
print("Model loaded on", device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

0_CLIPModel/pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Model loaded on cpu


In [ ]:
# ────────────────────────────────────────────────
# ❺ 主要関数
# ────────────────────────────────────────────────
def generate_clip_embeddings(images_dir: str, model):
    exts = {'.jpg', '.jpeg', '.png', '.webp', '.bmp', '.gif'}
    image_paths = sorted(str(p) for p in Path(images_dir).rglob('*') if p.suffix.lower() in exts)
    if not image_paths:
        raise ValueError(f'No images found under {images_dir}')

    images = [Image.open(p).convert('RGB') for p in image_paths]
    embeddings = model.encode(
        images, convert_to_numpy=True, normalize_embeddings=True,
        batch_size=32, show_progress_bar=True
    ).astype(np.float32)
    return embeddings, image_paths


def create_faiss_index(embeddings, paths, index_path):
    d = embeddings.shape[1]
    index = faiss.IndexFlatIP(d)
    index = faiss.IndexIDMap(index)
    index.add_with_ids(embeddings, np.arange(len(paths)))
    faiss.write_index(index, index_path)
    with open(index_path + '.paths', 'w') as f:
        f.writelines(p + '\n' for p in paths)
    print(f'✅ Index saved to {index_path}')
    return index


def load_faiss_index(index_path):
    index = faiss.read_index(index_path)
    with open(index_path + '.paths') as f:
        paths = [l.strip() for l in f]
    print(f'✅ Index loaded ({len(paths)} items)')
    return index, paths


def retrieve_similar(query, model, index, paths, top_k=TOP_K):
    if isinstance(query, str) and query.lower().endswith(('.png','.jpg','.jpeg','.webp','.bmp','.gif')):
        query_img = Image.open(query).convert('RGB')
        q_emb = model.encode(query_img, convert_to_numpy=True, normalize_embeddings=True)
    else:
        query_img = None
        q_emb = model.encode(query, convert_to_numpy=True, normalize_embeddings=True)

    q_emb = q_emb.astype(np.float32).reshape(1, -1)
    D, I = index.search(q_emb, top_k)
    return query_img, [paths[int(i)] for i in I[0]]


def visualize(query_img, retrieved_paths):
    n = len(retrieved_paths) + (1 if query_img else 0)
    plt.figure(figsize=(4*n, 4))
    col = 1
    if query_img:
        plt.subplot(1, n, col); col += 1
        plt.imshow(query_img); plt.axis('off'); plt.title('Query')
    for i, p in enumerate(retrieved_paths, 1):
        plt.subplot(1, n, col); col += 1
        plt.imshow(Image.open(p)); plt.axis('off')
        plt.title(f'Result {i}\n{Path(p).name}')
    plt.show()


In [ ]:
# ────────────────────────────────────────────────
# ❻ インデックスを作成 (初回のみ) またはロード
# ────────────────────────────────────────────────
if Path(INDEX_PATH).exists():
    index, img_paths = load_faiss_index(INDEX_PATH)      # ← 既存ファイルを使う
else:
    embeddings, img_paths = generate_clip_embeddings(IMAGES_PATH, model)
    index = create_faiss_index(embeddings, img_paths, INDEX_PATH)


Batches:   0%|          | 0/276 [00:00<?, ?it/s]

✅ Index saved to /content/drive/MyDrive/faiss_indexes/vector.index


In [ ]:
# ── バッチ検索：テスト用フォルダ内の全画像で実行 ──
from pathlib import Path

TESTDATA_PATH = '/content/drive/MyDrive/test'        # ★検索したい画像フォルダ
exts = {'.jpg', '.jpeg', '.png', '.webp', '.bmp', '.gif'}

query_paths = sorted(str(p) for p in Path(TESTDATA_PATH).rglob('*')
                     if p.suffix.lower() in exts)

if not query_paths:
    raise ValueError(f'No image files found under {TESTDATA_PATH}')

for q in query_paths:
    query_img, results = retrieve_similar(q, model, index, img_paths)

    print(f'▶ Query: {Path(q).name}')
    print('   Top-k:', [Path(p).name for p in results])
    visualize(query_img, results)          # 画像＋ファイル名を表示
